# ***Section Spectra Dimension Reduction***

This notebook shows the process of section spectra dimension reduction.

### ***Import packages***

Before we begin, let"s import all the necessary packages for this notebook.
First we add the directory which has our python files:

In [1]:
import sys

sys.path.insert(0, "../..")

Next we import all the necessary packages for this notebook:

In [2]:
import gc
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from typing import Tuple
from tensorflow.keras import optimizers
from tensorflow.keras import losses
from tensorflow.keras import callbacks
from tensorflow.keras import backend as K
from sklearn.model_selection import train_test_split
from skimage import (filters)
from tqdm import tqdm
from pyimzml.ImzMLParser import ImzMLParser, getionimage
from nnbiopsy.dimension_reduction.bn_vae import BNVAE

### ***Constants definitions***

Next, let's define some constant variables for this notebook:

In [3]:
# Define folder that contains the dhg dataset and files
DHG_PATH = "C:/Users/Leor/Desktop/Thesis/DHG"
# Define folder that contains the preprocessed dhg dataset
DATA_IN_PATH = f"{DHG_PATH}/Preprocessed"
# Define folder to save lower dimension data
DATA_OUT_PATH = f"{DHG_PATH}/Dimension_Reduction"
# Define file that contains dhg clinical state annotations
CLINICAL_STATE_ANNOTATIONS_PATH = f"{DHG_PATH}/Clinical_state_annotations.csv"
# Define folder to save VAE models for later use
MODELS_PATH = "C:/Users/Leor/Desktop/Thesis/section_vae_models"
# VAE model number of epochs
EPHOCS = 100
# VAE model batch size
BATCH_SIZE = 256
# VAE model intermediate layer size
INTERMEDIATE_LAYER_SIZE = 512
# VAE model latent layer size
LATENT_LAYER_SIZE = 5
# VAE model learning rate
LEARNING_RATE = 1e-3
# MSI Spectra dimension
SPECTRA_DIM = 92000
# The MSI sample type for filtering
SAMPLE_TYPE = "s"
# Mz value to get in order to threshold for tissue
TRESH_MZ = 750
# Mz tolerance value to get in order to threshold for tissue
TRESH_MZ_TOL = 150
# Treshould standard deviation for Gaussian kernel
TRESH_GAUSSIAN_SIGMA = 0.5

### ***Reading MSI clinical state anotations***

Next, let's read the clinical state anotations for each MSI:

In [4]:
# Read clinical state annotations csv
clinical_state_df = pd.read_csv(CLINICAL_STATE_ANNOTATIONS_PATH)

# Filter by sample_type
clinical_state_df = clinical_state_df[clinical_state_df["sample_type"] ==
                                      SAMPLE_TYPE]

### ***Get all spectra's from all MSI:***

Next, let's get all information except intensities (which needs a lot of memory) for each spectra from all MSI:

In [5]:
# Create lists to store each spectra's info
file_names = []
sample_numbers = []
histologies = []
who_grades = []
x_coordinates = []
y_coordinates = []
idxs = []
is_tissue = []

# Loop over each MSI
for index, msi_row in tqdm(clinical_state_df.iterrows(),
                           total=clinical_state_df.shape[0],
                           desc="MSI Loop"):
  # Parse the MSI file
  with ImzMLParser(os.path.join(DATA_IN_PATH,
                                f"{msi_row.file_name}.imzML")) as reader:
    # Get local TIC image of msi in mz region [600, 900]
    local_tic_img = getionimage(reader, TRESH_MZ, tol=TRESH_MZ_TOL)

    # Threshold image to separate tissue spectra from background
    smooth = filters.gaussian(local_tic_img, sigma=TRESH_GAUSSIAN_SIGMA)
    thresh_mean = filters.threshold_mean(smooth)
    thresh_img = local_tic_img > thresh_mean

    # Loop over each spectra
    for idx, (x, y, z) in tqdm(enumerate(reader.coordinates),
                               total=len(reader.coordinates),
                               desc="Spectra Loop"):
      # Keep sample file name of spectra
      file_names.append(msi_row.file_name)
      # Keep sample number of spectra
      sample_numbers.append(msi_row.sample_number)
      # Keep sample histology of spectra
      histologies.append(msi_row.histology)
      # Keep sample who grade of spectra
      who_grades.append(msi_row.who_grade)
      # Keep x coordinate of spectra
      x_coordinates.append(x)
      # Keep y coordinate of spectra
      y_coordinates.append(y)
      # Keep  of spectra
      idxs.append(idx)
      # Check if spectra is tissue
      if thresh_img[y - 1, x - 1]:
        # Keep boolean indication on if spectra is tissue
        is_tissue.append(True)
      else:
        # Keep boolean indication on if spectra is not tissue
        is_tissue.append(False)

# Convert to numpy array
file_names = np.array(file_names)
sample_numbers = np.array(sample_numbers)
histologies = np.array(histologies)
who_grades = np.array(who_grades)
x_coordinates = np.array(x_coordinates)
y_coordinates = np.array(y_coordinates)
idxs = np.array(idxs)
is_tissue = np.array(is_tissue)

MSI Loop: 100%|██████████| 24/24 [01:57<00:00,  4.92s/it]


### ***MSI parsers opening:***

Next, let's create parser for each MSI in order to read spectra's for the model:

In [6]:
# Opening parsers
parsers = {
    file_name: ImzMLParser(os.path.join(DATA_IN_PATH, f"{file_name}.imzML"))
    for file_name in clinical_state_df.file_name.unique()
}

### ***Dataset generator:***

Next, let's create a dataset generator for the model:

In [7]:
def map_index(index: tf.Tensor) -> Tuple[np.ndarray, np.ndarray]:
  """Function to map index to model input (spectra) and output (spectra).

  Args:
      index (tf.Tensor): index to map to corresponding values.

  Returns:
      Tuple[np.ndarray, np.ndarray]: input (spectra) and output (spectra).
  
  """
  # Decoding index from the EagerTensor object
  index = index.numpy()
  # Reading spectra from parser
  file_name = file_names[index]
  idx = idxs[index]
  _, spectra = parsers[file_name].getspectrum(idx)
  # Return spectra twice as input and reconstruction
  return (spectra, spectra)


def _fixup_shape(x: tf.Tensor, y: tf.Tensor):
  """ Function to Fix the implicit inferring of the shapes of the
  output Tensors.

  Args:
      x (tf.Tensor): input (spectra)
      y (tf.Tensor): output (spectra)

  Returns:
      Tuple[np.ndarray, np.ndarray]: input (spectra) and output (spectra) with
        correct shape.
  
  """
  x.set_shape([SPECTRA_DIM])
  y.set_shape([SPECTRA_DIM])
  return x, y


def create_ds(indexes: np.ndarray,
              batch_size: int,
              shuffle: bool = True) -> tf.data.Dataset:
  """Function to create a dataset for model

  Args:
      indexes (np.ndarray): indexes of thh dataset
      batch_size (int): batch size

  Returns:
      tf.data.Dataset: dataset
  """
  # Create dataset from generator
  ds = tf.data.Dataset.from_tensor_slices(indexes)
  # Shuffle the data
  if shuffle:
    ds = ds.shuffle(len(indexes))
  # Map index to spectra
  ds = ds.map(lambda i: tf.py_function(
      func=map_index, inp=[i], Tout=[tf.float32, tf.float32]))
  # Fix the implicit inferring of the shapes of the
  # output Tensors
  ds = ds.map(_fixup_shape)
  # Batch the spectra's
  ds = ds.batch(batch_size)
  # Prefetch batchs to make sure that a batch is ready to
  # be served at all time
  ds = ds.prefetch(tf.data.AUTOTUNE)
  return ds

### ***LOOCV Dimension reduction:***

Next, let"s apply dimension reduction using LOOCV for best evaluation:

In [8]:
train_metrics = {}
validation_metrics = {}

# Loop over each sample number
for exclude_sample in tqdm(np.unique(sample_numbers)):
  # Clear graph
  K.clear_session()
  gc.collect()

  # Create filter for training data - does not include the excluded sample
  # and includes tissue spectra's
  train_filter = ((sample_numbers != exclude_sample) & is_tissue)

  # Get indexes of all data
  indexes = np.arange(len(sample_numbers))

  # Get indexes of training data
  train_indexes = indexes[train_filter]

  # Get indexes of training and validation data
  train_indexes, val_indexes = train_test_split(
      train_indexes,
      test_size=0.2,
      random_state=0,
      stratify=is_tissue[train_filter])

  # Create data generators
  train_generator = create_ds(train_indexes, BATCH_SIZE)
  validation_generator = create_ds(val_indexes, BATCH_SIZE)

  # Create Callback to save the best model
  checkpoint_filepath = os.path.join(MODELS_PATH, f"excluded_{exclude_sample}/")
  model_checkpoint_callback = callbacks.ModelCheckpoint(
      filepath=checkpoint_filepath,
      save_weights_only=True,
      monitor="val_loss",
      mode="min",
      save_best_only=True)

  # Create Callback for model early stopping
  model_es_callback = callbacks.EarlyStopping(monitor='val_loss',
                                              mode='min',
                                              verbose=1,
                                              patience=5,
                                              min_delta=1e-10)

  # Create batch normalization VAE model
  vae_model = BNVAE(SPECTRA_DIM, INTERMEDIATE_LAYER_SIZE, LATENT_LAYER_SIZE)

  # Compile the model
  vae_model.compile(optimizers.Adam(learning_rate=LEARNING_RATE),
                    loss=losses.BinaryCrossentropy())

  # Train the model
  history = vae_model.fit(
      x=train_generator,
      validation_data=validation_generator,
      epochs=EPHOCS,
      callbacks=[model_checkpoint_callback, model_es_callback])

  # Load the best saved
  vae_model = BNVAE(SPECTRA_DIM, INTERMEDIATE_LAYER_SIZE, LATENT_LAYER_SIZE)
  vae_model.compile(loss=losses.BinaryCrossentropy())
  vae_model.load_weights(checkpoint_filepath).expect_partial()

  # Evaluate on train and validation
  train_metrics[exclude_sample] = vae_model.evaluate(x=train_generator)
  validation_metrics[exclude_sample] = vae_model.evaluate(
      x=validation_generator)

  # Clean model for next iteration
  vae_model = None

  # Separate training
  print("#" * 30)

  0%|          | 0/22 [00:00<?, ?it/s]

Epoch 1/100
84/84 [==============================] - 27s 300ms/step - loss: 0.1208 - val_loss: 0.1863
Epoch 2/100
84/84 [==============================] - 24s 286ms/step - loss: 0.0113 - val_loss: 0.0576
Epoch 3/100
84/84 [==============================] - 24s 290ms/step - loss: 0.0051 - val_loss: 0.0177
Epoch 4/100
84/84 [==============================] - 24s 288ms/step - loss: 0.0030 - val_loss: 0.0059
Epoch 5/100
84/84 [==============================] - 24s 284ms/step - loss: 0.0020 - val_loss: 0.0027
Epoch 6/100
84/84 [==============================] - 24s 288ms/step - loss: 0.0014 - val_loss: 0.0015
Epoch 7/100
84/84 [==============================] - 24s 283ms/step - loss: 0.0011 - val_loss: 9.8360e-04
Epoch 8/100
84/84 [==============================] - 26s 307ms/step - loss: 8.6534e-04 - val_loss: 8.1780e-04
Epoch 9/100
84/84 [==============================] - 24s 288ms/step - loss: 7.3096e-04 - val_loss: 6.4098e-04
Epoch 10/100
84/84 [==============================] - 26s 310m

  5%|▍         | 1/22 [23:35<8:15:18, 1415.15s/it]

##############################
Epoch 1/100
83/83 [==============================] - 26s 305ms/step - loss: 0.1220 - val_loss: 0.1865
Epoch 2/100
83/83 [==============================] - 24s 283ms/step - loss: 0.0115 - val_loss: 0.0597
Epoch 3/100
83/83 [==============================] - 24s 282ms/step - loss: 0.0050 - val_loss: 0.0186
Epoch 4/100
83/83 [==============================] - 24s 291ms/step - loss: 0.0031 - val_loss: 0.0061
Epoch 5/100
83/83 [==============================] - 23s 280ms/step - loss: 0.0020 - val_loss: 0.0027
Epoch 6/100
83/83 [==============================] - 25s 296ms/step - loss: 0.0015 - val_loss: 0.0016
Epoch 7/100
83/83 [==============================] - 24s 288ms/step - loss: 0.0011 - val_loss: 0.0011
Epoch 8/100
83/83 [==============================] - 26s 304ms/step - loss: 9.1217e-04 - val_loss: 8.5128e-04
Epoch 9/100
83/83 [==============================] - 25s 298ms/step - loss: 7.3288e-04 - val_loss: 6.7269e-04
Epoch 10/100
83/83 [===============

  9%|▉         | 2/22 [1:06:35<11:40:14, 2100.72s/it]

...

##############################
Epoch 1/100
82/82 [==============================] - 25s 289ms/step - loss: 0.1227 - val_loss: 0.1882
Epoch 2/100
82/82 [==============================] - 25s 302ms/step - loss: 0.0116 - val_loss: 0.0613
Epoch 3/100
82/82 [==============================] - 24s 295ms/step - loss: 0.0052 - val_loss: 0.0187
Epoch 4/100
82/82 [==============================] - 23s 282ms/step - loss: 0.0031 - val_loss: 0.0067
Epoch 5/100
82/82 [==============================] - 23s 273ms/step - loss: 0.0021 - val_loss: 0.0028
Epoch 6/100
82/82 [==============================] - 24s 286ms/step - loss: 0.0015 - val_loss: 0.0016
Epoch 7/100
82/82 [==============================] - 24s 292ms/step - loss: 0.0011 - val_loss: 0.0010
Epoch 8/100
82/82 [==============================] - 24s 295ms/step - loss: 8.9758e-04 - val_loss: 8.8925e-04
Epoch 9/100
82/82 [==============================] - 23s 274ms/step - loss: 7.6023e-04 - val_loss: 7.0063e-04
Epoch 10/100
82/82 [===============

 95%|█████████▌| 21/22 [13:48:27<37:22, 2242.21s/it]  

##############################
Epoch 1/100
79/79 [==============================] - 24s 288ms/step - loss: 0.1272 - val_loss: 0.1990
Epoch 2/100
79/79 [==============================] - 22s 277ms/step - loss: 0.0122 - val_loss: 0.0680
Epoch 3/100
79/79 [==============================] - 24s 296ms/step - loss: 0.0057 - val_loss: 0.0220
Epoch 4/100
79/79 [==============================] - 22s 270ms/step - loss: 0.0033 - val_loss: 0.0074
Epoch 5/100
79/79 [==============================] - 23s 282ms/step - loss: 0.0022 - val_loss: 0.0033
Epoch 6/100
79/79 [==============================] - 24s 307ms/step - loss: 0.0016 - val_loss: 0.0017
Epoch 7/100
79/79 [==============================] - 24s 302ms/step - loss: 0.0012 - val_loss: 0.0012
Epoch 8/100
79/79 [==============================] - 23s 292ms/step - loss: 9.7549e-04 - val_loss: 9.0164e-04
Epoch 9/100
79/79 [==============================] - 25s 317ms/step - loss: 7.9660e-04 - val_loss: 7.6323e-04
Epoch 10/100
79/79 [===============

100%|██████████| 22/22 [14:31:03<00:00, 2375.63s/it]

##############################


In [9]:
train_metrics

{2: 0.00016371734091080725,
 3: 0.00014856478082947433,
 4: 0.00014894203923176974,
 5: 0.00015434397209901363,
 6: 0.00014838961942587048,
 7: 0.00014829085557721555,
 8: 0.0001500753132859245,
 9: 0.00014811188157182187,
 10: 0.00014891465252730995,
 11: 0.0001494268944952637,
 12: 0.00015239159984048456,
 13: 0.00015565977082587779,
 14: 0.0001498508790973574,
 15: 0.00014876363275106996,
 16: 0.00014925902360118926,
 18: 0.0001554920308990404,
 19: 0.00015840522246435285,
 20: 0.00014949326578062028,
 21: 0.0001544183469377458,
 23: 0.0001538456417620182,
 25: 0.00015158245514612645,
 29: 0.0001497559860581532}

In [10]:
validation_metrics

{2: 0.000162978918524459,
 3: 0.00014925262075848877,
 4: 0.00014914802159182727,
 5: 0.000153902277816087,
 6: 0.00014869343431200832,
 7: 0.0001480912760598585,
 8: 0.0001497053017374128,
 9: 0.00014863813703414053,
 10: 0.0001489076530560851,
 11: 0.0001497933699283749,
 12: 0.00015307452122215182,
 13: 0.00015504025213886052,
 14: 0.00015001752763055265,
 15: 0.0001485478860558942,
 16: 0.00014939121319912374,
 18: 0.0001557704235892743,
 19: 0.00015819970576558262,
 20: 0.0001492952578701079,
 21: 0.00015416827227454633,
 23: 0.00015409440675284714,
 25: 0.00015146390069276094,
 29: 0.00014951439516153187}

### ***Leave one sample out dimension reduction - tissue spectra's:***

Next, let's get the dimension reduction of each of left out sample spectra's:

In [11]:
# Define list to store new lower dimension data
lower_dimension_data = []

# Loop over each sample number
for exclude_sample in tqdm(np.unique(sample_numbers)):
  # Clear graph
  K.clear_session()
  gc.collect()

  # Create filter for test data
  test_filter = ((sample_numbers == exclude_sample) & is_tissue)

  # Get indexes of all data
  indexes = np.arange(len(sample_numbers))

  # Create test data generator
  test_generator = create_ds(indexes[test_filter], BATCH_SIZE, False)

  # Get saved model path
  model_path = os.path.join(MODELS_PATH, f"excluded_{exclude_sample}/")

  # Load model
  vae_model = BNVAE(SPECTRA_DIM, INTERMEDIATE_LAYER_SIZE, LATENT_LAYER_SIZE)
  vae_model.compile(loss=losses.BinaryCrossentropy())
  vae_model.load_weights(checkpoint_filepath).expect_partial()

  # Evaluate on test set
  vae_model.evaluate(test_generator)

  # Get encoded features
  for sample, i in zip(test_generator.unbatch(), indexes[test_filter]):
    new_sample = vae_model.encoder(np.expand_dims(
        sample[0], axis=0))[2].numpy()[0].tolist()
    new_sample.insert(0, y_coordinates[i])
    new_sample.insert(0, x_coordinates[i])
    new_sample.insert(0, idxs[i])
    new_sample.insert(0, sample_numbers[i])
    new_sample.insert(0, file_names[i])
    new_sample.append(histologies[i])
    new_sample.append(who_grades[i])
    lower_dimension_data.append(new_sample)

  # Clean model for next iteration
  vae_model = None

# Create data frame and save it
columns = [
    "file_name", "sample_number", "idx", "x_coordinate", "y_coordinate"
] + [f"F{i}" for i in range(0, LATENT_LAYER_SIZE)] + ["histology", "who_grade"]
lower_dimension_df = pd.DataFrame(lower_dimension_data, columns=columns)

lower_dimension_df.to_csv(f"{DATA_OUT_PATH}/section-tissue.csv", index=False)

  0%|          | 0/22 [00:00<?, ?it/s]

3/3 [==============================] - 1s 302ms/step - loss: 1.4609e-04


  5%|▍         | 1/22 [00:04<01:33,  4.46s/it]

4/4 [==============================] - 1s 185ms/step - loss: 1.5094e-04


  9%|▉         | 2/22 [00:09<01:36,  4.85s/it]

...

 91%|█████████ | 20/22 [02:01<00:13,  6.85s/it]

5/5 [==============================] - 1s 184ms/step - loss: 1.5018e-04


 95%|█████████▌| 21/22 [02:07<00:06,  6.62s/it]

9/9 [==============================] - 2s 189ms/step - loss: 1.5204e-04


100%|██████████| 22/22 [02:18<00:00,  6.28s/it]


### ***Leave one sample out dimension reduction - non tissue spectra's:***

Next, let's get the dimension reduction of each of left out sample spectra's:

In [12]:
# Define list to store new lower dimension data
lower_dimension_data = []

# Loop over each sample number
for exclude_sample in tqdm(np.unique(sample_numbers)):
  # Clear graph
  K.clear_session()
  gc.collect()

  # Create filter for test data
  test_filter = ((sample_numbers == exclude_sample) & ~is_tissue)

  # Get indexes of all data
  indexes = np.arange(len(sample_numbers))

  # Create test data generator
  test_generator = create_ds(indexes[test_filter], BATCH_SIZE, False)

  # Get saved model path
  model_path = os.path.join(MODELS_PATH, f"excluded_{exclude_sample}/")

  # Load model
  vae_model = BNVAE(SPECTRA_DIM, INTERMEDIATE_LAYER_SIZE, LATENT_LAYER_SIZE)
  vae_model.compile(loss=losses.BinaryCrossentropy())
  vae_model.load_weights(checkpoint_filepath).expect_partial()

  # Evaluate on test set
  vae_model.evaluate(test_generator)

  # Get encoded features
  for sample, i in zip(test_generator.unbatch(), indexes[test_filter]):
    new_sample = vae_model.encoder(np.expand_dims(
        sample[0], axis=0))[2].numpy()[0].tolist()
    new_sample.insert(0, y_coordinates[i])
    new_sample.insert(0, x_coordinates[i])
    new_sample.insert(0, idxs[i])
    new_sample.insert(0, sample_numbers[i])
    new_sample.insert(0, file_names[i])
    new_sample.append(histologies[i])
    new_sample.append(who_grades[i])
    lower_dimension_data.append(new_sample)

  # Clean model for next iteration
  vae_model = None

# Create data frame and save it
columns = [
    "file_name", "sample_number", "idx", "x_coordinate", "y_coordinate"
] + [f"F{i}" for i in range(0, LATENT_LAYER_SIZE)] + ["histology", "who_grade"]
lower_dimension_df = pd.DataFrame(lower_dimension_data, columns=columns)

lower_dimension_df.to_csv(f"{DATA_OUT_PATH}/section-non-tissue.csv", index=False)

  0%|          | 0/22 [00:00<?, ?it/s]

3/3 [==============================] - 1s 265ms/step - loss: 1.4985e-04


  5%|▍         | 1/22 [00:04<01:26,  4.11s/it]

5/5 [==============================] - 2s 269ms/step - loss: 1.5035e-04


  9%|▉         | 2/22 [00:10<01:50,  5.52s/it]

...

 91%|█████████ | 20/22 [02:41<00:15,  7.68s/it]

6/6 [==============================] - 2s 257ms/step - loss: 1.4870e-04


 95%|█████████▌| 21/22 [02:48<00:07,  7.59s/it]

9/9 [==============================] - 3s 245ms/step - loss: 1.4823e-04


100%|██████████| 22/22 [02:59<00:00,  8.16s/it]


### ***MSI parsers closing:***

Next, let"s close MSI parsers:

In [13]:
# Closing parsers
for reader in parsers.values():
  if reader.m:
    reader.m.close()